In [ ]:
import math
import csv
import folium

from geopy.distance import geodesic

In [ ]:
def get_hexagon_row_coord(start_coord, radius, count, coord_data):
    if count== 0:
        return coord_data
    else:
        next_coord = geodesic(meters=radius*2).destination(start_coord, 90)
        row = {
            'coordinate': (next_coord.latitude, next_coord.longitude),
        }
        coord_data.append(row)
        return get_hexagon_row_coord(next_coord, radius, count-1, coord_data)

In [ ]:
def row_mapper(start_coord, radius, row_count, height_count, coord_data):
    height = radius * math.sqrt(3)
    if height_count == 0:
        return coord_data
    elif height_count % 2 != 0:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 90
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    else:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 270
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    return row_mapper(next_coord, radius, row_count, height_count-1, coord_data)

In [ ]:
def data_to_csv(data):
    file_path = 'coordinates/hexagonal-packing-v1.csv'
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)
        print(f"Coordinates saved to {file_path}")

In [ ]:
r = 500
start_point = (-8.429957624116502, 115.03252775649212)
row_count = 50
height_count = 17
coordinates = row_mapper(start_point, r, row_count, height_count, [])

In [ ]:
print(coordinates)

In [ ]:
data_to_csv(coordinates)

In [ ]:
world_map = folium.Map(location=start_point, zoom_start=15)

for coord in coordinates:
    folium.Marker(
        location=coord['coordinate'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(world_map)

In [ ]:
world_map.save('test-map.html')